In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

from wordcloud import WordCloud

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from src import utils, nlp_topic_utils, web_app_utils

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\morga\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


SyntaxError: invalid syntax (web_app_utils.py, line 6)

# Data Visualizations for Website

## Import Data

In [ ]:
wallstreet = pd.read_csv('data/wallstreet_master.csv', engine='python')

In [ ]:
wallstreet.head()

## Daily Forum Posts

In [ ]:
def plot_daily_post_freq(series):
    '''
    Interactive line plot showing the number of plots per day.
    '''
    daily_post_freq = pd.DataFrame(series.value_counts()).reset_index()
    daily_post_freq.columns = ['date', 'count']
    daily_post_freq.sort_values(['date'], axis=0, inplace=True)
    fig = px.line(data_frame=daily_post_freq, x='date', y='count', labels= {'date': 'Date', 'count': '# of Posts'})
    fig.update_layout(width=1500, height=600)
    fig.show();

In [ ]:
series = wallstreet['post_date_only']

web_app_utils.plot_daily_post_freq(series)

## Wordcloud

In [ ]:
def display_wordcloud(pd_series, max_words):
    '''
    Generates wordcloud.
    '''

    series_as_str = pd_series.astype('str')
    joined_wordcloud_text = ' '.join(str_series)

    wordcloud = WordCloud(background_color='white', max_words=max_words, contour_color='steelblue')
    wordcloud.generate(joined_wordcloud_text)

    plt.figure(figsize=(20,7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off');

In [ ]:
display_wordcloud(wallstreet['wordcloud_text'], 10000)

## Sentiment Time-Series Analysis

In [ ]:
si_analyzer = SentimentIntensityAnalyzer()
wallstreet['compound_sentiment'] = wallstreet['wordcloud_text'].apply(lambda x: si_analyzer.polarity_scores(x)['compound'])

In [ ]:
arr1 = wallstreet['sentiment_all']

# utils.stats_summary(arr1)

In [ ]:
# def plot_sentiment_time_series(sentiment_arr)

groupby1 = wallstreet.groupby('post_date_only')['compound_sentiment'].mean()
daily_sentiment = pd.DataFrame(groupby1).reset_index()
daily_sentiment.columns=['date', 'sentiment']
fig = px.line(data_frame=daily_sentiment, x='date', y='sentiment')#, labels= {'date': 'Date', 'count': '# of Posts'})
fig.show();


In [ ]:
wallstreet_for_sentiment = wallstreet[wallstreet['post_date_only'] > '2017-10-14'].copy()

arr2 = wallstreet_for_sentiment['sentiment_all']
sentiment_lt_avg = wallstreet_for_sentiment['sentiment_all'].mean()

utils.stats_summary(arr2)

In [ ]:
daily_sentiment = wallstreet_for_sentiment.groupby('post_date_only')['compound_sentiment'].aggregate(['min', 'max', 'mean', 'median', 'std', 'count']).reset_index()
daily_sentiment.columns = ['date', 'min', 'max', 'mean', 'median', 'std_dev', 'count']

daily_sentiment['less_1_std_dev'] = daily_sentiment['mean'] - daily_sentiment['std_dev']
daily_sentiment['plus_1_std_dev'] = daily_sentiment['mean'] + daily_sentiment['std_dev']

In [ ]:
date_rng = daily_sentiment['date'].values
date_rng_rev = date_rng[::-1]

lifetime_avg_arr = np.full(len(date_rng), sentiment_lt_avg)
mean_arr = daily_sentiment['mean'].values
median_arr = daily_sentiment['median'].values

upper_lim_arr = daily_sentiment['plus_1_std_dev'].values
lower_lim_arr = daily_sentiment['less_1_std_dev'].values
lower_lim_arr = lower_lim_arr[::-1]

max_arr = daily_sentiment['max'].values
min_arr = daily_sentiment['min'].values


fig = go.Figure()
fig.update_layout(width=1800, height=700,yaxis=dict(range=[-1.1,1.1]))

#fig.add_trace(go.Scatter(x=date_rng, y=median_arr, name='Lifetime Avg.', line=dict(color='#FFA15A', width=2)))
#fig.add_trace(go.Scatter(x=date_rng, y=median_arr, name='Daily Median', line=dict(color='#B6E880', width=2)))
#fig.add_trace(go.Scatter(x=date_rng, y=mean_arr, name='Daily Avg.', line=dict(color='royalblue', width=2)))


fig.add_trace(go.Scatter(x=date_rng, y=upper_lim_arr+lower_lim_arr, fill='toself', fillcolor='rgba(231,107,243,0.2)',
                         line_color='rgba(255,255,255,0)', name='One Std. Dev. Range'))

# fig.add_trace(go.Scatter(x=date_rng, y=lower_lim_arr, name='Lower Lim', stackgroup='one', line=dict(color='firebrick', width=0.5)))
# fig.add_trace(go.Scatter(x=date_rng, y=upper_lim_arr, name='Upper Lim', stackgroup='one', line=dict(color='firebrick', width=0.5)))


fig.add_trace(go.Scatter(x=date_rng, y=min_arr, name='Min', line=dict(color='green', width=2, dash='dot')))
fig.add_trace(go.Scatter(x=date_rng, y=max_arr, name='Max', line=dict(color='green', width=2, dash='dot')))
fig.show();

In [ ]:

fig, ax = plt.subplots(figure(20,8))
# Plot record 2005 to 2014 daily temperature range.
ax.plot(date_rng, tmp_max, linestyle = '-', color = 'green', linewidth = 1)
plt.plot(days, tmp_min, linestyle = '-', color = 'yellow', linewidth = 1)
plt.fill_between(days, tmp_max, tmp_min, color='grey', alpha=0.2)
# Plot the 2015 temperatures that were greater than 2005 to 2014 maximum temperature.
plt.plot(days, tmax_15_grtr, 'ro')
# Plot the 2015 temperatures that were less than 2005 to 2014 minimum temperature.
plt.plot(days, tmin_15_less, 'bo')

In [ ]:
daily_sentiment_v2['std_dev'].hist(bins=40)